In [1]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [3]:
import sys
sys.path.append('..')

from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "，", "。", "【", ","])


In [4]:
from tools.graph_builder import TwlfGraphBuilder
graph.query('MATCH (n) DETACH DELETE n;')

graph_builder = TwlfGraphBuilder(graph)

dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(dir, filename))
        doc_pages = loader.load()
        graph_builder.graph_build(doc_pages, text_splitter)

In [5]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)
graph_store = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])

In [6]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import AzureChatOpenAI
import os
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)
graph.refresh_schema()
graph_tradition_chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
graph_tradition_tool = graph_tradition_chain.as_tool()
graph_tradition_tool


/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


StructuredTool(name='GraphCypherQAChain', description="Takes {'title': 'ChainInput', 'type': 'object', 'properties': {'query': {'title': 'Query'}}}.", args_schema=<class 'pydantic.v1.main.ChainInput'>, func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x16885c540>, coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x16885c5e0>)

In [7]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9}
)
retriever

VectorStoreRetriever(tags=['Neo4jVector', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x137745190>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9})

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    "你是一個有用的助手, 你的任務是回答問題."
    "你必須根據以下提供的檢索內容及資料庫查詢結果進行問答問題."
    "如果檢索內容為空, 則回答 '沒有找到相關資訊'"
    "以 5 至 10 句話以內回應, 保持答案的簡潔"
    "以下為檢索內容:\n\n"
    "{context}"

    "以下為資料庫查詢結果:\n\n"
    "{tradition_result}"

    問題: {question}
    """
)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "tradition_result": graph_tradition_chain}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
question = "未滿15歲被保險人死亡，身故保險金該怎麼賠?"

In [11]:
retriever.invoke(question)

[Document(metadata={'source_idx': 'a2240e6e-8cbd-43db-9a3f-783e8c023223', 'chunk_idx': 1, 'name': 'Chunk_1', 'page_num': 1}, page_content='\ncontent: ，本公司應退還歷年累計儲存生息\n之金額予要保人。  \n本公司於每一保 單年度屆滿後，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保\n人。 \n \n【身故保險金或喪葬費用保險金的給付】  \n第十六條  \n被保險人於本契約有效期間內身故者，本公司按身故日之當年度保險金額給付身故保險金。  \n訂立本契約時，以未滿 15足歲之未成年人為被保險人，除喪葬費用之給付外，其餘死亡給付之\n約定於被保險人滿 15足歲之日起發生效力；被保險人滿 15足歲前死亡者，其身故保險金變更為\n喪葬費用保險金。  \n前項未滿 15足歲之被保險人如有於民國九十九年二月三日 (不含 )前訂立之保險契約，其喪葬費\n用保險金之給付依下列方式辦理 ：'),
 Document(metadata={'source_idx': '7171a441-7f44-4922-a4d6-8445c296e3ca', 'chunk_idx': 1, 'name': 'Chunk_1', 'page_num': 1}, page_content='\ncontent: ，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保人。  \n \n【身故保險金或喪 葬費用保險金的給付】  \n第十五條 \n被保險人於本契約有效期間內身故者，本公司按下列三款取其最大值給付身故保險金：  \n一、身故日之當年度保險金額。  \n二、身故日之保單價值準備金 乘以保單價值準備金比率所得之金額 。 \n三、身故日之應繳保險費總和。  \n訂立本契約時，以未滿 15足歲之未成年人為被保險人，除喪葬費用之給付外，其餘死亡給付之約定\n於被保險人滿 15足歲之日起發生效力；被保險人滿 15足歲前死亡者，其身故保險金變更為喪葬費用\n保險金。  \n前項未滿 15足歲之被保險人如有於民國九十九年二月三日 (不含)前訂立之保險契約，其喪葬費用保\n險金之給付依下列方式辦理：  \n一、被保險人於民國九十九年二月三日 (不含

In [12]:
graph_tradition_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:PART]->(c:Chunk)
WHERE c.content CONTAINS '未滿15歲被保險人死亡，身故保險金該怎麼賠?'
RETURN d, c

Full Context:
[]

> Finished chain.


{'query': '未滿15歲被保險人死亡，身故保險金該怎麼賠?', 'result': '我不知道答案。'}

In [13]:
rag_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:PART]->(c:Chunk)
WHERE c.content CONTAINS '未滿15歲被保險人死亡，身故保險金該怎麼賠?'
RETURN d.filename, c.page_num, c.content

Full Context:
[]

> Finished chain.


'根據提供的檢索內容，未滿15歲的被保險人死亡時，並不適用一般的身故保險金給付。相反，依照保險契約，這些情況下的身故保險金會變更為喪葬費用保險金。若被保險人是在民國九十九年二月三日（不含）前訂立的保險契約，喪葬費用保險金的給付方式會依照相關遺產稅法的規定進行調整。具體而言，喪葬費用保險金額若超過一定限額，保險公司不負給付責任，並應無息退還超過部分的已繳保險費。'